In [1]:
%pip install ipynb
%pip install python-dotenv transformers accelerate bitsandbytes --quiet

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from ipynb.fs.full.recommendations import *

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
C:\Users\Josue\AppData\Roaming\Python\Python312\site-packages\bitsandbytes\nn\modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


 Based on your current state and preferences, here are three recommended activities:

1. Go for a run: This will provide you with some exercise while enjoying the beautiful weather. Running has numerous benefits for both physical and mental health.
2. Listen to nature sounds: If you prefer indoor activities, listen to some calming nature sounds like birds chirping or waves crashing. These sounds can help reduce stress and improve focus.
3. Do yoga: Yoga combines mindfulness and relaxation techniques with low-impact exercises, making it perfect for someone like you who wants to stay active but also unwind after a busy day.


In [3]:
from ipynb.fs.full.emotion_classification_model import *


Is Cuda Available: True
Cuda Version: 12.4


Generating train split: 0 examples [00:00, ? examples/s]

{0: 'admiration', 1: 'amusement', 2: 'anger', 3: 'annoyance', 4: 'approval', 5: 'caring', 6: 'confusion', 7: 'curiosity', 8: 'desire', 9: 'disappointment', 10: 'disapproval', 11: 'disgust', 12: 'embarrassment', 13: 'excitement', 14: 'fear', 15: 'gratitude', 16: 'grief', 17: 'joy', 18: 'love', 19: 'nervousness', 20: 'optimism', 21: 'pride', 22: 'realization', 23: 'relief', 24: 'remorse', 25: 'sadness', 26: 'surprise', 27: 'neutral'}
{'admiration': 0, 'amusement': 1, 'disapproval': 10, 'disgust': 11, 'embarrassment': 12, 'excitement': 13, 'fear': 14, 'gratitude': 15, 'grief': 16, 'joy': 17, 'love': 18, 'nervousness': 19, 'anger': 2, 'pride': 21, 'realization': 22, 'relief': 23, 'remorse': 24, 'sadness': 25, 'surprise': 26, 'neutral': 27, 'annoyance': 3, 'approval': 4, 'caring': 5, 'confusion': 6, 'curiosity': 7, 'desire': 8, 'disappointment': 9, 'optimism': 20}


Map:   0%|          | 0/4485 [00:00<?, ? examples/s]

Map:   0%|          | 0/1122 [00:00<?, ? examples/s]

Map:   0%|          | 0/4485 [00:00<?, ? examples/s]

Map:   0%|          | 0/1122 [00:00<?, ? examples/s]

Counter({19: 1783, 25: 963, 27: 874, 6: 865})



c:\Users\Josue\CS-GY-6923 Machine Learning\synap-well\synap-well\notebooks\emotion_classification_model.ipynb:144: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  "\n",


tensor([[-8.2014, -5.4721, -5.4850, -5.9139, -5.3802, -5.4057, -5.6049, -3.0468,
         -0.2606, -2.0675, -5.4187, -3.8337, -3.2218, -3.4117, -1.4994, -6.9996,
         -3.9410, -5.2227, -7.5257, -0.8615, -4.1418, -4.9742, -3.8780, -3.9929,
         -5.9717, -3.7598, -3.7755, -4.5145]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
8
Predicted Emotion: desire
tensor([[2.5253e-03, 4.5607e-04, 5.9474e-04, 7.8180e-04, 1.1512e-02, 9.5014e-03,
         5.5871e-04, 6.6844e-04, 5.8952e-04, 3.0818e-02, 2.9249e-04, 9.6733e-03,
         6.3770e-03, 2.1248e-03, 1.9716e-02, 3.7302e-04, 4.9179e-02, 3.7416e-03,
         3.3376e-04, 1.4382e-01, 3.7564e-04, 9.6262e-03, 4.5922e-03, 3.6339e-02,
         1.5353e-03, 6.4790e-01, 1.1208e-03, 4.8769e-03]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>)
25
Predicted emotion: sadness
Predicted emotion: Negative
Emotion Classification Result:
  Superclass: Negative
  Sub-emotion: disappointment (0.326)
  Top 5 Emotions:
    - disappointment: 0.3259


In [14]:
import pandas as pd

# Load your dataset
wellness_df = pd.read_csv("../data/processed/processed_wellness_data.csv")

# Columns that represent the tags
tag_columns = [
    'activity_level_grouping',
    'demographic_health_grouping',
    'sleep_health',
    'heart_health',
    'bp_health',
    'stress_profile_grouping',
    'social_wellness_grouping'
]

# Bundle the tags per row
# Removes any NaN/null values
wellness_df['tags_per_row'] = wellness_df[tag_columns].apply(
    lambda row: [tag for tag in row if pd.notna(tag)],
    axis=1
)

# Convert each list of tags into a comma-separated string
wellness_df['tags_per_row_str'] = wellness_df['tags_per_row'].apply(lambda tags: ', '.join(tags))

# View the result
print(wellness_df[['tags_per_row_str']].head())

## STATE VALUE for recommendation workflow
first_tags_str = wellness_df.loc[wellness_df.index[0], 'tags_per_row_str']
print(first_tags_str)

                                    tags_per_row_str
0  UNKNOWN, UNHEALTHY_CONSUMPTION, GOOD_SLEEP, GO...
1  UNKNOWN, UNHEALTHY_CONSUMPTION, POOR_SLEEP, GO...
2  UNKNOWN, UNHEALTHY_CONSUMPTION, GOOD_SLEEP, GO...
3  UNKNOWN, UNHEALTHY_CONSUMPTION, GOOD_SLEEP, GO...
4  UNKNOWN, UNHEALTHY_CONSUMPTION, POOR_SLEEP, GO...
UNKNOWN, UNHEALTHY_CONSUMPTION, GOOD_SLEEP, GOOD_HEART_RATE, POOR_BLOOD_PRESSURE,  HIGH_STRESS , HIGH_SOCIAL


In [5]:
from dotenv import load_dotenv
import os
import requests

# Load Environment Variables
load_dotenv()

# Hugging Face Token
HUGGING_FACE_TOKEN = os.getenv("HUGGING_FACE_TOKEN")

#Weather API Token
WEATHER_API_KEY = os.getenv("WEATHER_API_KEY")

In [7]:
# Read the Initial Questions csv file into a DataFrame
initial_df = pd.read_csv('../data/raw/CS-GY-6923 Log Tracker  - Initial Questions.csv')

# Extract the last row of the DataFrame
last_updated_entry = initial_df.iloc[-1]

# De-structure relevant information from features into variables
gender = last_updated_entry['Gender']
age = last_updated_entry['Age']
height = last_updated_entry['Height']
activities = last_updated_entry['Activities for Mental Health (such as therapy sessions or meditation) [Comma separated]']
quality_social_interactions = last_updated_entry['Quality or frequency of social interactions']
location = last_updated_entry['Location']

print(gender)
print(age)
print(height)
print(activities)
print(quality_social_interactions)
print(location)

Male
25
5'9"
Watching Movies, Gaming, Sleeping
Moderate, Daily
Brooklyn, NY


In [9]:
# Read the Daily Questions csv file into a DataFrame
daily_df = pd.read_csv('../data/raw/CS-GY-6923 Log Tracker  - Daily Questions.csv')

# # Read the Processed Daily Questions csv file into a DataFrame
# daily_df = pd.read_csv('../data/processed/CS-GY-6923 Log Tracker  - Daily Questions.csv')

# # Columns that represent the tags
# tag_columns = [
#     'activity_level_grouping',
#     'demographic_health_grouping',
#     'sleep_health',
#     'heart_health',
#     'bp_health',
#     'stress_profile_grouping',
#     'social_wellness_grouping'
# ]

# # Bundle the tags per row
# # Removes any NaN/null values
# daily_df['tags_per_row'] = daily_df[tag_columns].apply(
#     lambda row: [tag for tag in row if pd.notna(tag)],
#     axis=1
# )

# # Convert each list of tags into a comma-separated string
# daily_df['tags_per_row_str'] = daily_df['tags_per_row'].apply(lambda tags: ', '.join(tags))

# # View the result
# print(daily_df[['tags_per_row_str']].head())

# Extract the first row of the DataFrame
first_daily_entry = daily_df.iloc[0]

# De-structure relevant information from features into variables
notes = first_daily_entry['Notes']
# age = first_daily_entry['Age']
# height = first_daily_entry['Height']
# activities = first_daily_entry['Activities for Mental Health (such as therapy sessions or meditation) [Comma separated]']
# quality_social_interactions = first_daily_entry['Quality or frequency of social interactions']
# location = first_daily_entry['Location']

print(notes)
# print(age)
# print(height)
# print(activities)
# print(quality_social_interactions)
# print(location)

Barely had any sleep, I am feeling terrible


In [10]:
# Weather Prompt
from datetime import datetime

def fetch_weather_data(city_name, api_key):
    """
    Fetch current weather description from WeatherAPI for a given city.
    """
    url = f'http://api.weatherapi.com/v1/current.json?key={api_key}&q={city_name}'
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()

        if 'current' in data and 'condition' in data['current']:
            return data['current']['condition']['text']
        else:
            print("Warning: Weather data structure is incomplete.")
            return None

    except requests.exceptions.RequestException as e:
        print(f"Error fetching weather data: {e}")
        return None
    
def fetch_past_weather_data(city_name, date, api_key):
    """
    Fetch current weather description from WeatherAPI for a given city.
    """   

    raw_date = date  # MM/DD/YYYY format
    date_obj = datetime.strptime(raw_date, '%m/%d/%Y')
    formatted_date = date_obj.strftime('%Y-%m-%d')
    
    url = f'http://api.weatherapi.com/v1/history.json?key={api_key}&q={city_name}&dt={formatted_date}'
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()

        print(data)

        if 'forecast' in data and 'forecastday' in data['forecast'] and len(data['forecast']['forecastday']) > 0 and 'day' in data['forecast']['forecastday'][0] and 'condition' in data['forecast']['forecastday'][0]['day']:
            return data['forecast']['forecastday'][0]['day']['condition']['text']
        else:
            print("Warning: Weather data structure is incomplete.")
            return None

    except requests.exceptions.RequestException as e:
        print(f"Error fetching weather data: {e}")
        return None


def classify_weather_situation(weather_description):
    """
    Classify weather condition into one of: SUNNY, RAINY, CLOUDY, WINDY, or UNKNOWN.
    """
    description = weather_description.lower()

    if 'sunny' in description:
        return 'SUNNY'
    elif 'rain' in description or 'shower' in description:
        return 'RAINY'
    elif 'overcast' in description or 'cloud' in description:
        return 'CLOUDY'
    elif 'wind' in description:
        return 'WINDY'
    else:
        return 'UNKNOWN'

api_key = WEATHER_API_KEY

city_name = location

# weather_description = fetch_weather_data(city_name, api_key) #Commented this out to save API calls (Limited to 30,000 Per Month I believe)
# weather_description = fetch_past_weather_data(city_name, "04/26/2025", api_key) #Commented this out to save API calls (Limited to 30,000 Per Month I believe)


# if weather_description:
#     weather_situation = classify_weather_situation(weather_description)
#     print(f"City: {city_name}")
#     print(f"Weather Description: {weather_description}")
#     print(f"Classified Weather Situation: {weather_situation}")
# else:
#     print("Unable to fetch or classify weather data.")



In [11]:
# Preferences Prompt

preferences = activities

In [20]:
# Resulting Classification probability output from Regressor_Predictive_Model File
import torch
import time

regressor_probs = torch.Tensor([0.3,0.3, 0.4])

daily_activity_recommendation_list = []

for index, row in daily_df.iterrows():
    if index > 5:
        break
    notes = row['Notes']  # Get the note for this row
    date = row['Date'] # Get the date for this row
    # mental_body_state_tags = row['tags_per_row_str']
    mental_body_state_tags = wellness_df.loc[wellness_df.index[0], 'tags_per_row_str']
    print(mental_body_state_tags)

    # Assume regressor_probs is available
    regressor_probs = torch.Tensor([0.3,0.3, 0.4])

    # features_for_regressor = row[["duration_of_sleep_hours", "quality_of_sleep_110", "level_of_stress_scale_110", 
    #                             "level_of_workload_scale_110"]]    

    # # Assume regressor_probs is available
    # # Assumes that RegressorModel is already loaded into the variables

    # regressor_probs = invoke_regressor(features_for_regressor)
    
    # # assuming regressor is not defined as a global variable, but it should be in the context of your notebook
    # regressor_probs = invoke_Regressor(regressor_model,features_for_regressor)

    # Run the function on each note
    result = run_emotion_pipeline(notes, regressor_probs)

    print("Emotion Classification Result:")
    print(f"  {notes}")
    print(f"  Superclass: {result['superclass']}")
    print(f"  Sub-emotion: {result['sub_emotion']} ({result['confidence']:.3f})")
    # print("  Top 5 Emotions:")
    # for label, prob in result["top_5_emotions"]:
    #     print(f"    - {label}: {prob:.4f}")

    superclass_emotion = result['superclass']
    sub_emotion = result['sub_emotion']
    mood_string = f"{superclass_emotion} - {sub_emotion}"

    weather_description = fetch_past_weather_data(location, date, WEATHER_API_KEY) 
    print(weather_description)
    # time.sleep(1)

    weather_situation = classify_weather_situation(weather_description)
    print(weather_situation)
    
    daily_recommended_activities= generate_activity_suggestions(mood_string, weather_situation, mental_body_state_tags, preferences)

    generated_prompt_parts = daily_recommended_activities.split("[/INST]")

    generated_prompt_question = generated_prompt_parts[0]
    generated_prompt_answer = generated_prompt_parts[1]
    
    daily_activity_recommendation_list.append(f"{generated_prompt_question}:"
                                              f"    {generated_prompt_answer}")


# result = run_emotion_pipeline(notes, regressor_probs)

# print("Emotion Classification Result:")
# print(f"  Superclass: {result['superclass']}")
# print(f"  Sub-emotion: {result['sub_emotion']} ({result['confidence']:.3f})")
# print("  Top 5 Emotions:")
# for label, prob in result["top_5_emotions"]:
#     print(f"    - {label}: {prob:.4f}")


UNKNOWN, UNHEALTHY_CONSUMPTION, GOOD_SLEEP, GOOD_HEART_RATE, POOR_BLOOD_PRESSURE,  HIGH_STRESS , HIGH_SOCIAL
Emotion Classification Result:
  Barely had any sleep, I am feeling terrible
  Superclass: Negative
  Sub-emotion: nervousness (0.284)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{'location': {'name': 'Brooklyn', 'region': 'New York', 'country': 'United States of America', 'lat': 40.65, 'lon': -73.95, 'tz_id': 'America/New_York', 'localtime_epoch': 1745730111, 'localtime': '2025-04-27 01:01'}, 'forecast': {'forecastday': [{'date': '2025-03-14', 'date_epoch': 1741910400, 'day': {'maxtemp_c': 9.6, 'maxtemp_f': 49.3, 'mintemp_c': 2.8, 'mintemp_f': 37.0, 'avgtemp_c': 6.0, 'avgtemp_f': 42.8, 'maxwind_mph': 9.2, 'maxwind_kph': 14.8, 'totalprecip_mm': 0.0, 'totalprecip_in': 0.0, 'totalsnow_cm': 0.0, 'avgvis_km': 10.0, 'avgvis_miles': 6.0, 'avghumidity': 83, 'daily_will_it_rain': 0, 'daily_chance_of_rain': 0, 'daily_will_it_snow': 0, 'daily_chance_of_snow': 0, 'condition': {'text': 'Overcast', 'icon': '//cdn.weatherapi.com/weather/64x64/day/122.png', 'code': 1009}, 'uv': 2.0}, 'astro': {'sunrise': '07:09 AM', 'sunset': '07:02 PM', 'moonrise': '07:38 PM', 'moonset': '07:18 AM', 'moon_phase': 'Full Moon', 'moon_illumination': 100}, 'hour': [{'time_epoch': 1741924800, 'ti

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{'location': {'name': 'Brooklyn', 'region': 'New York', 'country': 'United States of America', 'lat': 40.65, 'lon': -73.95, 'tz_id': 'America/New_York', 'localtime_epoch': 1745730168, 'localtime': '2025-04-27 01:02'}, 'forecast': {'forecastday': [{'date': '2025-03-15', 'date_epoch': 1741996800, 'day': {'maxtemp_c': 7.2, 'maxtemp_f': 44.9, 'mintemp_c': 5.1, 'mintemp_f': 41.2, 'avgtemp_c': 6.2, 'avgtemp_f': 43.1, 'maxwind_mph': 10.7, 'maxwind_kph': 17.3, 'totalprecip_mm': 0.6, 'totalprecip_in': 0.02, 'totalsnow_cm': 0.0, 'avgvis_km': 10.0, 'avgvis_miles': 6.0, 'avghumidity': 93, 'daily_will_it_rain': 1, 'daily_chance_of_rain': 100, 'daily_will_it_snow': 0, 'daily_chance_of_snow': 0, 'condition': {'text': 'Patchy rain possible', 'icon': '//cdn.weatherapi.com/weather/64x64/day/176.png', 'code': 1063}, 'uv': 2.0}, 'astro': {'sunrise': '07:07 AM', 'sunset': '07:03 PM', 'moonrise': '08:38 PM', 'moonset': '07:36 AM', 'moon_phase': 'Waning Gibbous', 'moon_illumination': 99}, 'hour': [{'time_epo

...

{'location': {'name': 'Brooklyn', 'region': 'New York', 'country': 'United States of America', 'lat': 40.65, 'lon': -73.95, 'tz_id': 'America/New_York', 'localtime_epoch': 1745730249, 'localtime': '2025-04-27 01:04'}, 'forecast': {'forecastday': [{'date': '2025-03-17', 'date_epoch': 1742169600, 'day': {'maxtemp_c': 13.5, 'maxtemp_f': 56.3, 'mintemp_c': 5.0, 'mintemp_f': 41.0, 'avgtemp_c': 10.5, 'avgtemp_f': 50.9, 'maxwind_mph': 15.9, 'maxwind_kph': 25.6, 'totalprecip_mm': 32.7, 'totalprecip_in': 1.29, 'totalsnow_cm': 0.0, 'avgvis_km': 7.8, 'avgvis_miles': 4.0, 'avghumidity': 93, 'daily_will_it_rain': 1, 'daily_chance_of_rain': 100, 'daily_will_it_snow': 0, 'daily_chance_of_snow': 0, 'condition': {'text': 'Moderate or heavy rain shower', 'icon': '//cdn.weatherapi.com/weather/64x64/day/356.png', 'code': 1243}, 'uv': 3.0}, 'astro': {'sunrise': '07:04 AM', 'sunset': '07:05 PM', 'moonrise': '10:41 PM', 'moonset': '08:16 AM', 'moon_phase': 'Waning Gibbous', 'moon_illumination': 93}, 'hour': 

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{'location': {'name': 'Brooklyn', 'region': 'New York', 'country': 'United States of America', 'lat': 40.65, 'lon': -73.95, 'tz_id': 'America/New_York', 'localtime_epoch': 1745730294, 'localtime': '2025-04-27 01:04'}, 'forecast': {'forecastday': [{'date': '2025-03-18', 'date_epoch': 1742256000, 'day': {'maxtemp_c': 15.0, 'maxtemp_f': 59.1, 'mintemp_c': 0.9, 'mintemp_f': 33.6, 'avgtemp_c': 7.4, 'avgtemp_f': 45.3, 'maxwind_mph': 14.8, 'maxwind_kph': 23.8, 'totalprecip_mm': 0.0, 'totalprecip_in': 0.0, 'totalsnow_cm': 0.0, 'avgvis_km': 10.0, 'avgvis_miles': 6.0, 'avghumidity': 58, 'daily_will_it_rain': 0, 'daily_chance_of_rain': 0, 'daily_will_it_snow': 0, 'daily_chance_of_snow': 0, 'condition': {'text': 'Sunny', 'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png', 'code': 1000}, 'uv': 5.0}, 'astro': {'sunrise': '07:02 AM', 'sunset': '07:06 PM', 'moonrise': '11:44 PM', 'moonset': '08:40 AM', 'moon_phase': 'Waning Gibbous', 'moon_illumination': 87}, 'hour': [{'time_epoch': 1742270400, 

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{'location': {'name': 'Brooklyn', 'region': 'New York', 'country': 'United States of America', 'lat': 40.65, 'lon': -73.95, 'tz_id': 'America/New_York', 'localtime_epoch': 1745730336, 'localtime': '2025-04-27 01:05'}, 'forecast': {'forecastday': [{'date': '2025-03-19', 'date_epoch': 1742342400, 'day': {'maxtemp_c': 10.8, 'maxtemp_f': 51.4, 'mintemp_c': 4.9, 'mintemp_f': 40.9, 'avgtemp_c': 7.8, 'avgtemp_f': 46.0, 'maxwind_mph': 11.0, 'maxwind_kph': 17.6, 'totalprecip_mm': 0.0, 'totalprecip_in': 0.0, 'totalsnow_cm': 0.0, 'avgvis_km': 10.0, 'avgvis_miles': 6.0, 'avghumidity': 79, 'daily_will_it_rain': 0, 'daily_chance_of_rain': 0, 'daily_will_it_snow': 0, 'daily_chance_of_snow': 0, 'condition': {'text': 'Overcast', 'icon': '//cdn.weatherapi.com/weather/64x64/day/122.png', 'code': 1009}, 'uv': 3.0}, 'astro': {'sunrise': '07:00 AM', 'sunset': '07:07 PM', 'moonrise': 'No moonrise', 'moonset': '09:09 AM', 'moon_phase': 'Waning Gibbous', 'moon_illumination': 80}, 'hour': [{'time_epoch': 174235

In [ ]:
for string in daily_activity_recommendation_list:
    print(string)
# print(daily_activity_recommendation_list)

["[INST] The user is feeling Negative - nervousness, the weather is CLOUDY, and they are currently in a state of UNKNOWN, UNHEALTHY_CONSUMPTION, GOOD_SLEEP, GOOD_HEART_RATE, POOR_BLOOD_PRESSURE,  HIGH_STRESS , HIGH_SOCIAL. They typically enjoy activities like Watching Movies, Gaming, Sleeping. Based on the weather, user's mood, state, and preferences, suggest at most 3 relevant (preferably fitness-focused for at least 1 activity) activities with a 50-word short description for each one, that would produce a healthy mental well-being in the user. :     Activity 1: Yoga Movie Night\nDescription: Escape to a peaceful world by watching yoga movies while practicing some gentle stretches. This will help you relax your body and mind after a long day.\r\n\r\nActivity 2: Virtual Fitness Classes\r\nDescription: Work out from the comfort of your home through virtual classes. These classes cater to all fitness levels and can be done indoors or outdoors depending on the weather.\r\n\r\nActivity 3: 